## 4日目

### Functor則
* 第一法則: idでファンクター値を写した場合、ファンクター値が変化してはいけない
    - id => 単位元の事だと思う。当然だね

In [3]:
import scalaz._, Scalaz._

import scalaz._, Scalaz._

In [2]:
List(1, 2, 3) map {identity} assert_=== List(1, 2, 3)

* 第二法則: f と g の合成関数でファンクター値を写したものと、まず g, 次に f でファンクター値を写したもの が等しい

In [3]:
(List(1, 2, 3) map {{(_: Int) * 3} map {(_: Int) + 1}}) assert_=== (List(1, 2, 3) map {(_: Int) * 3} map {(_: Int) + 1})

* 最近はREPLでFunctor則のチェックしてくれるらしい。便利。

In [6]:
sealed trait COption[+A] {}
case class CSome[A](counter: Int, a: A) extends COption[A]
case object CNone extends COption[Nothing]

implicit def coptionEqual[A]: Equal[COption[A]] = Equal.equalA
implicit val coptionFunctor = new Functor[COption] {
    def map[A, B](fa: COption[A])(f: A => B): COption[B] = fa match {
        case CNone => CNone
        case CSome(c, a) => CSome(c + 1, f(a))
    }
}

defined trait COption
defined class CSome
defined object CNone
defined function coptionEqual
coptionFunctor: AnyRef with Functor[COption] = $sess.cmd5Wrapper$Helper$$anon$1@4eb48540

In [7]:
(CSome(0, "ho"): COption[String]) map {(_: String) + "ha"}

res6: COption[String] = CSome(1,hoha)

In [8]:
(CSome(0, "ho"): COption[String]) map {identity}

res7: COption[String] = CSome(1,ho)

* jupyterでのやり方がわからなかったのでsbt consoleで実行

```scala
scala> functor.laws[COption].check
! functor.invariantFunctor.identity: Falsified after 0 passed tests.
> ARG_0: CSome(0,0)
! functor.invariantFunctor.composite: Falsified after 0 passed tests.
> ARG_0: CSome(0,559927601)
> ARG_1: <function1>
> ARG_2: <function1>
> ARG_3: <function1>
> ARG_4: <function1>
! functor.identity: Falsified after 0 passed tests.
> ARG_0: CSome(0,1)
! functor.composite: Falsified after 0 passed tests.
> ARG_0: CSome(0,1959572567)
> ARG_1: <function1>
> ARG_2: <function1>
```

### Applicative則

```scala
trait ApplicativeLaw extends FunctorLaw {
    def identityAp[A](fa: F[A])(implicit FA: Equal[F[A]]): Boolean =
      FA.equal(ap(fa)(point((a: A) => a)), fa)
    
    def composition[A, B, C](fbc: F[B => C], fab: F[A => B], fa: F[A])(implicit FC: Equal[F[C]]) =
      FC.equal(ap(ap(fa)(fab))(fbc), ap(fa)(ap(fab)(ap(fbc)(point((bc: B => C) => (ab: A => B) => bc compose ab)))))
    
    def homomorphism[A, B](ab: A => B, a: A)(implicit FB: Equal[F[B]]): Boolean =
      FB.equal(ap(point(a))(point(ab)), point(ab(a)))
    
    def interchange[A, B](f: F[A => B], a: A)(implicit FB: Equal[F[B]]): Boolean =
      FB.equal(ap(point(a))(f), ap(f)(point((f: A => B) => f(a))))
}
```

* 2引数以上の関数の入ったFunctorと他のFunctorを作用させるためのFunctorの拡張らしい

### Semigroup則

```scala
trait SemigroupLaw {
    def associative(f1: F, f2: F, f3: F)(implicit F: Equal[F]): Boolean =
      F.equal(append(f1, append(f2, f3)), append(append(f1, f2), f3))
}
```

* 結合律と同じってことかな

### Monoid則

```scala
trait MonoidLaw extends SemigroupLaw {
  def leftIdentity(a: F)(implicit F: Equal[F]) = F.equal(a, append(zero, a))
  def rightIdentity(a: F)(implicit F: Equal[F]) = F.equal(a, append(a, zero))
}
```

* 単位元を左右のどちらに `|+|` しても同じ値が返ってくるということ

In [4]:
1 * 2 assert_=== 2

In [5]:
2 * 1 assert_=== 2

### Monoid としての Option

In [6]:
implicit def optionMonoid[A: Semigroup]: Monoid[Option[A]] = new Monoid[Option[A]] {
    def append(f1: Option[A], f2: => Option[A]) = (f1, f2) match {
        case (Some(a1), Some(a2)) => Some(Semigroup[A].append(a1, a2))
        case (Some(a1), None) => f1
        case (None, Some(a2)) => f2
        case (None, None) => None
    }
    
    def zero: Option[A] = None
}

defined function optionMonoid

In [7]:
(none: Option[String]) |+| "andy".some

res6: Option[String] = Some("andy")

In [8]:
(Ordering.LT: Ordering).some |+| none

res7: Option[Ordering] = Some(LT)

In [9]:
Tags.First('a'.some) |+| Tags.First('b'.some)

res8: Option[Char] @@ Tags.First = Some(a)

In [10]:
Tags.First(none:Option[Char]) |+| Tags.First('b'.some)

res9: Option[Char] @@ Tags.First = Some(b)

In [11]:
Tags.First('a'.some) |+| Tags.First(none: Option[Char])

res10: Option[Char] @@ Tags.First = Some(a)

In [12]:
Tags.Last('a'.some) |+| Tags.Last('b'.some)

res11: Option[Char] @@ Tags.Last = Some(b)

In [13]:
Tags.Last(none: Option[Char]) |+| Tags.Last('b'.some)

res12: Option[Char] @@ Tags.Last = Some(b)

In [14]:
Tags.Last('a'.some) |+| Tags.Last(none: Option[Char])

res13: Option[Char] @@ Tags.Last = Some(a)

### Foldable
* 畳み込み!!

In [15]:
List(1, 2, 3).foldRight (1) {_ * _}

res14: Int = 6

In [16]:
9.some.foldLeft(2) {_ + _}

res15: Int = 11

In [17]:
List(1, 2, 3) foldMap {identity}

res16: Int = 6

In [18]:
List(true, false, true, true) foldMap {Tags.Disjunction.apply}

res17: Boolean @@ Tags.Disjunction = true